In [1]:
import torch
import numpy as np
from PIL import Image, ImageSequence

from diffusion_wrapper import DiffusionWrapper
from heatmap_generator import HeatmapGenerator
from zero_shot_tracker import ZeroShotTracker


diffusion_wrapper = DiffusionWrapper('../text-to-video-ms-1.7b')
heatmap_generator = HeatmapGenerator()
zero_shot_tracker = ZeroShotTracker()

video_features_dict = diffusion_wrapper.extract_video_features('../rocket256.gif', "A rocket starting on Mars.")

/home/max/miniconda3/envs/diffusers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...:  40%|████      | 2/5 [00:00<00:00,  5.80it/s]2024-05-19 15:36:20.946762: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-19 15:36:20.979600: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 15:36:21.627239:

In [4]:
video_features = diffusion_wrapper.concatenate_video_features({'up_block': [video_features_dict['up_block'][2]]})
video_features = video_features.permute(0, 2, 3, 1).float().cpu()

In [5]:
heatmaps = heatmap_generator.generate(video_features, (93, 137, 0))
heatmap_generator.safe_heatmap_as_gif(heatmaps)

#heatmaps = torch.permute(heatmaps, (0, 3, 1, 2))
#heatmaps = torch.nn.functional.interpolate(heatmaps, size=256, mode="bilinear", align_corners=True) * 255
#
#heatmaps = heatmaps.squeeze().numpy()
#frames_gif = [Image.fromarray(f) for f in heatmaps]
#frames_gif[0].save("output/heatmaps.gif", save_all=True, append_images=frames_gif[1:], duration=100, loop=0)

tracks = zero_shot_tracker.track(heatmaps)

def load_frames(image: Image, mode='RGB'):
    return np.array([
        np.array(frame.convert(mode))
        for frame in ImageSequence.Iterator(image)
    ])

with Image.open('../rocket256.gif') as im:
    frames = load_frames(im)

zero_shot_tracker.place_marker_in_frames(frames, tracks)